In [1]:
import sys

sys.path.append("../../")

In [2]:
import numpy as np
import pandas as pd
import shared
import xarray as xr
from shared import open_dataarray, open_dataset, open_zarr, save

from pyCIAM import spherical_nearest_neighbor as snn

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


## Aggregate Sweet et al. datasets

In [3]:
# Global data
glob = (
    open_dataset(shared.DIR_SLR_SWEET_RAW / "TR_global_projections.nc")
    .rename(years="year", percentiles="quantile")
    .sel(year=slice(2020, None))[
        ["GMSL_Low", "GMSL_IntLow", "GMSL_Int", "GMSL_IntHigh", "GMSL_High"]
    ]
    .to_array(dim="scenario")
)
glob["scenario"] = glob.scenario.str[5:]
glob["quantile"] = glob["quantile"] / 100

In [4]:
# Gauge data
ds_loc = (
    open_dataset(shared.DIR_SLR_SWEET_RAW / "TR_local_projections.nc")
    .set_coords(["lon", "lat"])
    .rename(percentiles="quantile", years="year", tg="site_id")
    .dropna(how="all", dim="year")
    .sel(year=slice(2020, None))
)

# drop stations w/ QC issues
ds_loc = ds_loc.sel(site_id=~ds_loc.QC_flag.astype(bool))

# rearrange
ds_loc = ds_loc[[i for i in ds_loc.data_vars if i.startswith("rsl_total")]].to_array(
    dim="scenario"
)
ds_loc["quantile"] = ds_loc["quantile"] / 100
ds_loc["scenario"] = ds_loc.scenario.str[10:]
assert ds_loc.notnull().all()

# rename site ID as numeric
ds_loc["site_id"] = np.arange(ds_loc.site_id.size)

In [5]:
# Gridded data
grid = (
    open_dataset(shared.DIR_SLR_SWEET_RAW / "TR_gridded_projections.nc")
    .rename(percentiles="quantile", years="year")
    .dropna(how="all", dim="year")
    .sel(year=slice(2020, None))
    .stack(site_id=["lon", "lat"])
)

# rearrange
grid = grid[[i for i in grid.data_vars if i.startswith("rsl_total")]].to_array(
    dim="scenario"
)
grid["quantile"] = grid["quantile"] / 100
grid["scenario"] = grid.scenario.str[10:]

# drop land pixels that are not valid
grid = grid.sel(site_id=(grid != -31985).all(["scenario", "quantile", "year"]))
assert grid.notnull().all()

# rename site_id
grid = grid.reset_index("site_id").assign_coords(
    site_id=np.arange(ds_loc.site_id.size, ds_loc.site_id.size + grid.site_id.size)
)

In [6]:
# we call these "05" even though they are currently relative to '00. Will convert next
full = xr.Dataset(
    {
        "lsl_msl05": xr.concat((ds_loc, grid.interp(year=ds_loc.year)), "site_id"),
        "gsl_msl05": glob,
    }
)
assert full.lsl_msl05.notnull().all()

### Adjust to 2005 baseline

First the LSL

In [7]:
adj = (
    open_dataarray(shared.PATH_SLR_HIST_TREND_MAP).rename(
        longitude="lon", latitude="lat"
    )
    * 5
)
adj["lon"] = adj.lon.where(adj.lon <= 180, adj.lon - 360)
adj = adj.to_dataframe().dropna().reset_index()

full_sites = full[["lon", "lat"]].to_dataframe()
full_sites["lon"] = full_sites.lon.where(full_sites.lon <= 180, full_sites.lon - 360)

vals = adj.loc[snn(full_sites, adj.reset_index()).values, "sea_level_trends"]
vals.index = pd.Index(full.site_id.values, name="site_id")

full["lsl_msl05"] -= vals.to_xarray()

Then the GSL

In [8]:
adj = open_dataset(shared.PATH_SLR_GMSL_HIST_TIMESERIES).msl.to_series()
full["gsl_msl05"] -= (
    adj.loc["1996":"2014"].mean() - adj.loc["1991":"2009"].mean()
) * 1000

### Update to meters

In [9]:
full.lon.attrs.clear()
full.lat.attrs.clear()
full = (
    full.pint.quantify(lsl_msl05="mm", gsl_msl05="mm")
    .pint.to(lsl_msl05="meters", gsl_msl05="meters")
    .pint.dequantify()
)

### Add NCC scenario

In [11]:
# use AR6 VLM scenario for Sweet

ar6 = open_zarr(shared.PATH_SLR_AR6)[["lon", "lat", "lsl_ncc_msl05"]]

aligned_sites = snn(
    full_sites, ar6[["lon", "lat"]].to_dataframe().reset_index(drop=True)
).values
full["lsl_ncc_msl05"] = (
    ar6.lsl_ncc_msl05.sel(quantile=full["quantile"])
    .isel(site_id=aligned_sites)
    .interp(year=full.year)
    .assign_coords(site_id=full.site_id)
)

### Add description

In [12]:
full.scenario.attrs.update(
    {
        "description": (
            "Scenarios are defined by their 2100 GMSL change relative to 2000. "
            "Low=0.3m; IntLow=0.5m; Int=1.0m; IntHigh=1.5m; High=2.0m"
        )
    }
)

## Save

In [13]:
for v in full.variables:
    full[v].encoding.clear()

In [14]:
save(full.chunk({"site_id": 100}), shared.PATH_SLR_SWEET, mode="w")